In [1]:
import arcpy
import json

# 指定 geodatabase 路徑
gdb = r"D:\ArcGIS\Morpho_Spatial_Demography\MSD_Taiwan.gdb"

# 取得所有 domain
domains = arcpy.da.ListDomains(gdb)

# 建立輸出字典
domain_dict = {}

for domain in domains:
    if domain.domainType == "CodedValue":
        domain_dict[domain.name] = domain.codedValues

# 匯出 JSON
out_json = r"C:\Users\keelu\Downloads\domains.json"
with open(out_json, "w", encoding="utf-8") as f:
    json.dump(domain_dict, f, ensure_ascii=False, indent=4)

print("完成匯出：", out_json)


完成匯出： C:\Users\keelu\Downloads\domains.json


In [1]:
###欄位純文字轉domain code

import arcpy

# === 需要改的參數 ===
fc = r"D:\ArcGIS\Morpho_Spatial_Demography\MSD_Taiwan.gdb\OL_P3_2204_corals_taglab"  # 要更新的圖層/要素類別
text_field = "TL_Class"                        # 文字欄位（如：Favi、Gala…）
out_field  = "Genus_ID"                        # 目標數值欄位（Short/Long）
default_if_missing = None                      # 若無對應要寫什麼；可改成 0 或留 None

# 文字 → 代碼 對照表（依你附圖）
MAP = {
    "ACAN": 21, "ACRO": 4, "BRIA": 29, "CYPH": 5, "DIPS": 17,
    "ECHI": 25, "EUPH": 20, "FAVI": 10, "FUGI": 15, "GALA": 7,
    "GONO": 22, "HELI": 13, "HYDN": 6, "LEPT": 24, "LOBO": 14,
    "MERU": 26, "MILL": 12, "MONT": 3, "OTHER_OCT": 32,
    "OTHER_ORGANISM": 30, "OTHER_SCL": 31, "PACH": 19,
    "PAVO": 11, "PECT": 9, "PLAT": 8, "POCI": 1, "PORI": 2,
    "SARC": 28, "SERI": 23, "SINU": 27, "STYL": 18,
    "TURB": 16, "TURF": 33,
}

# 也支援你表內的大小寫/底線/空白差異
def norm(s):
    return (s or "").strip().upper()

# === 執行更新 ===
# 先檢查欄位是否存在
f_names = [f.name for f in arcpy.ListFields(fc)]
if text_field not in f_names:
    raise ValueError(f"找不到文字欄位 {text_field}")
if out_field not in f_names:
    raise ValueError(f"找不到目標欄位 {out_field}（請先建立數值欄位）")

cnt_ok = cnt_missing = 0
with arcpy.da.UpdateCursor(fc, [text_field, out_field]) as cur:
    for tl_text, cur_val in cur:
        key = norm(tl_text).replace(" ", "").replace("-", "")
        code = MAP.get(key, default_if_missing)
        if code is None:
            cnt_missing += 1
        else:
            cnt_ok += 1
        cur.updateRow((tl_text, code))

print(f"完成：寫入 {cnt_ok} 筆；無對應 {cnt_missing} 筆（已寫入 {default_if_missing}）。")


完成：寫入 423 筆；無對應 0 筆（已寫入 None）。


In [15]:
### 欄位有設定domain

import arcpy

# === 需要修改的參數 ===
fc = r"D:\ArcGIS\Morpho_Spatial_Demography\DL_training.gdb\k2b_2025_DL_sample"
text_field = "Forms"      # 已套用 Domain 的「文字」欄位
out_field  = "form_id"         # 要寫入的數值欄位（Short/Long）
domain_name = "Growth_form"           # 欄位使用的域名稱

# === 工具函式 ===
def norm(s):
    return ("" if s is None else str(s)).strip().upper().replace(" ", "").replace("_", "").replace("-", "")

def to_int_or_none(x):
    try:
        return int(x)
    except Exception:
        return None

# === 基本檢查 ===
ws = arcpy.Describe(fc).path
fmap = {f.name:(f.type, f.domain) for f in arcpy.ListFields(fc)}
if text_field not in fmap: raise ValueError(f"找不到欄位 {text_field}")

# 🔧 若無 out_field 則自動新增（預設 LONG）
if out_field not in fmap:
    arcpy.AddField_management(in_table=fc, field_name=out_field, field_type="LONG")
    # 重新讀取欄位資訊
    fmap = {f.name:(f.type, f.domain) for f in arcpy.ListFields(fc)}

#（原本的檢查保留，若新增失敗仍會拋錯）
if out_field  not in fmap: raise ValueError(f"找不到欄位 {out_field}（請先建立數值欄位）")

print(f"[INFO] {text_field} 型別={fmap[text_field][0]}、Domain={fmap[text_field][1]}")
print(f"[INFO] {out_field}  型別={fmap[out_field][0]}、Domain={fmap[out_field][1]}")

# === 讀取 Domain ===
code2desc = None
for d in arcpy.da.ListDomains(ws):
    if d.name == domain_name:
        code2desc = d.codedValues  # {code: desc}，code 可能是字串或整數
        break
if code2desc is None:
    raise ValueError(f"找不到名為 {domain_name} 的域")

# 構建 4 種索引：
# 1) 描述→代碼（標準化後）
desc2code = {norm(desc): code for code, desc in code2desc.items()}

# 2) 代碼字串→代碼（標準化後；這可比對欄位值若正好就是代碼字串）
codeStr2code = {norm(code): code for code in code2desc.keys()}

# 3) 代碼整數集合（若代碼能轉成 int）
intCodes = {int(code) for code in code2desc.keys() if to_int_or_none(code) is not None}

# 4) 代碼整數對照：整數→原始代碼（有時你想保留原代碼；我們下面直接用整數寫入）
intCode2orig = {int(code): code for code in code2desc.keys() if to_int_or_none(code) is not None}

print(f"[INFO] Domain {domain_name} - 總筆數: {len(code2desc)}")
# 顯示前幾筆檢查
for i, (k, v) in enumerate(code2desc.items()):
    print(f"  sample[{i}] code={k!r} ({type(k).__name__})  desc={v!r}")
    if i >= 4: break

# === 先看看欄位中有哪些不同值（少量預覽）
seen = set()
with arcpy.da.SearchCursor(fc, [text_field]) as cur:
    for (val,) in cur:
        seen.add(val)
        if len(seen) >= 8:
            break
print("[INFO] 欄位值樣本：", [repr(s) for s in list(seen)])

# === 逐列更新 ===
cnt_ok = cnt_missing = 0
with arcpy.da.UpdateCursor(fc, [text_field, out_field]) as cur:
    for tl_val, _ in cur:
        code_out = None
        key = norm(tl_val)

        # 1) 若欄位存「描述文字」，用描述→代碼
        if code_out is None:
            c = desc2code.get(key)
            if c is not None:
                # 若代碼可轉成整數就用整數，否則保持原型（若 out_field 是數值欄位，請確保能轉成整數）
                code_out = to_int_or_none(c) if to_int_or_none(c) is not None else c

        # 2) 若欄位存「代碼字串」，用代碼字串→代碼
        if code_out is None:
            c = codeStr2code.get(key)
            if c is not None:
                code_out = to_int_or_none(c) if to_int_or_none(c) is not None else c

        # 3) 若欄位看起來是數字，且剛好等於某個代碼整數
        if code_out is None and to_int_or_none(tl_val) is not None:
            as_int = int(tl_val)
            if as_int in intCodes:
                code_out = as_int

        # 4) 若仍找不到，放棄或給預設值（例如 0）
        #    這裡我保留 None；如果你想寫 0，就改成 code_out = 0
        if code_out is None:
            cnt_missing += 1
        else:
            cnt_ok += 1

        cur.updateRow((tl_val, code_out))

print(f"完成：寫入 {cnt_ok} 筆；無對應 {cnt_missing} 筆。")


[INFO] Forms 型別=String、Domain=Growth_form
[INFO] form_id  型別=Integer、Domain=
[INFO] Domain Growth_form - 總筆數: 11
  sample[0] code='1' (str)  desc='br'
  sample[1] code='2' (str)  desc='cb'
  sample[2] code='3' (str)  desc='di'
  sample[3] code='4' (str)  desc='co'
  sample[4] code='5' (str)  desc='ta'
[INFO] 欄位值樣本： ["'6'", "'4'", "'1'", "'7'", "'3'", "'2'"]
完成：寫入 573 筆；無對應 0 筆。


In [9]:
import arcpy

# === 需要改的參數 ===
fc = r"D:\ArcGIS\Morpho_Spatial_Demography\MSD_Hawaii.gdb\rr_1a_2023_coral"  # 要更新的 feature class
text_field = "species"   # 來源欄位（文字）
out_field  = "Forms"  # 目標欄位（數值或文字皆可）
default_if_missing = None

# === Description→Code 對照表 ===
MAP = {
    "br": 1,
    "cb": 2,
    "di": 3,
    "co": 4,
    "ta": 5,
    "ma": 6,
    "en": 7,
    "fr": 8,
    "fo": 9,
    "bu": 10,
    "su": 11
}

def norm(s):
    return (s or "").strip().lower()

# === 檢查欄位是否存在 ===
f_names = [f.name for f in arcpy.ListFields(fc)]
if text_field not in f_names:
    raise ValueError(f"找不到文字欄位 {text_field}")
if out_field not in f_names:
    raise ValueError(f"找不到目標欄位 {out_field}（請先建立該欄位）")

# === 更新欄位 ===
cnt_ok = cnt_missing = 0
codes_set = set()

with arcpy.da.UpdateCursor(fc, [text_field, out_field]) as cur:
    for txt, old_code in cur:
        key = norm(txt)
        code = MAP.get(key, default_if_missing)
        if code is None:
            cnt_missing += 1
        else:
            cnt_ok += 1
            codes_set.add(code)
        cur.updateRow((txt, code))

print(f"✅ 完成：成功寫入 {cnt_ok} 筆，無對應 {cnt_missing} 筆。")
print(f"共產生 {len(codes_set)} 種不同代碼：{sorted(codes_set)}")


✅ 完成：成功寫入 0 筆，無對應 578 筆。
共產生 0 種不同代碼：[]


In [3]:
##將文字欄位設定成domain

import arcpy

# === 要改的參數 ===
fc = r"D:\ArcGIS\Morpho_Spatial_Demography\MSD_Hawaii.gdb\rr_1a_2023_coral"
field = "Form"
domain_name = "Growth_form"

# === 將欄位設定為使用該 domain ===
arcpy.AssignDomainToField_management(fc, field, domain_name)
print(f"已將欄位 {field} 設定為使用 domain：{domain_name}")

# === 從 geodatabase 讀取 domain 對照 ===
desc = arcpy.Describe(fc)
gdb_path = desc.path
domain_list = arcpy.da.ListDomains(gdb_path)

domain = next((d for d in domain_list if d.name == domain_name), None)
if not domain:
    raise ValueError(f"找不到 domain：{domain_name}")

desc_to_code = {v.lower(): k for k, v in domain.codedValues.items()}

# === 更新欄位值 ===
cnt_ok = cnt_missing = 0
codes = set()

with arcpy.da.UpdateCursor(fc, [field]) as cur:
    for (val,) in cur:
        key = (val or "").strip().lower()
        code = desc_to_code.get(key)
        if code is not None:
            cur.updateRow([code])
            cnt_ok += 1
            codes.add(code)
        else:
            cnt_missing += 1

print(f"✅ 完成：已將文字轉換為 domain 代碼。")
print(f"成功轉換 {cnt_ok} 筆，未匹配 {cnt_missing} 筆。")
print(f"共使用 {len(codes)} 種代碼：{sorted(codes)}")


已將欄位 Form 設定為使用 domain：Growth_form
✅ 完成：已將文字轉換為 domain 代碼。
成功轉換 0 筆，未匹配 961 筆。
共使用 0 種代碼：[]


In [2]:
###數值ID轉換對應文字

import arcpy

# === 需要改的參數 ===
fc = r"D:\ArcGIS\Morpho_Spatial_Demography\MSD_Taiwan.gdb\OL_P1_2312_corals"  # 要更新的 feature class
num_field = "Form_ID"      # 來源欄位（數值）
out_field = "Forms"    # 目標欄位（文字）
default_if_missing = None

# === Code → Description 對照表 ===
MAP = {
    1: "br",
    2: "cb",
    3: "di",
    4: "co",
    5: "ta",
    6: "ma",
    7: "en",
    8: "fr",
    9: "fo",
    10: "bu",
    11: "su"
}

# === 檢查欄位是否存在，若沒文字欄位就建立 ===
f_names = [f.name for f in arcpy.ListFields(fc)]

if num_field not in f_names:
    raise ValueError(f"找不到來源欄位 {num_field}")

if out_field not in f_names:
    print(f"⚠️ 找不到文字欄位 {out_field}，正在建立...")
    arcpy.AddField_management(fc, out_field, "TEXT")
    print(f"✅ 已建立文字欄位 {out_field}")

# === 更新欄位 ===
cnt_ok = cnt_missing = 0
text_set = set()

with arcpy.da.UpdateCursor(fc, [num_field, out_field]) as cur:
    for num, old_txt in cur:
        txt = MAP.get(num, default_if_missing)
        if txt is None:
            cnt_missing += 1
        else:
            cnt_ok += 1
            text_set.add(txt)
        cur.updateRow((num, txt))

print(f"✅ 完成：成功寫入 {cnt_ok} 筆，無對應 {cnt_missing} 筆。")
print(f"共產生 {len(text_set)} 種不同文字：{sorted(text_set)}")


✅ 完成：成功寫入 4336 筆，無對應 0 筆。
共產生 7 種不同文字：['br', 'cb', 'di', 'en', 'fo', 'ma', 'su']
